In [1]:
import pandas as pd
import numpy as np
from google.colab import auth
from google.cloud import storage

auth.authenticate_user()

gcs = storage.Client()

file_path1 = 'gs://jpm_lstm_data/LSTM_prediction.csv'
file_path2 = 'gs://jpm_lstm_data/sperated_tapes.csv'

df_predict = pd.read_csv(file_path1)
df_v = pd.read_csv(file_path2)
print(df_predict.head())

     Unnamed: 0  Open  High  Low      Close
0  2025-04-28-1   158   160  140  152.05226
1  2025-04-28-2   152   161  144  152.44218
2  2025-04-28-3   158   162  142  154.09073
3  2025-04-28-4   161   162  141  155.02562
4  2025-04-28-5   160   160  140  155.11020


In [2]:
df_predict['Open'] = pd.to_numeric(df_predict['Open'], downcast='float')
df_predict['High'] = pd.to_numeric(df_predict['High'], downcast='float')
df_predict['Low'] = pd.to_numeric(df_predict['Low'], downcast='float')
df_predict

,Unnamed: 0,Open,High,Low,Close
0,2025-04-28-1,158.0,160.0,140.0,152.052260
1,2025-04-28-2,152.0,161.0,144.0,152.442180
2,2025-04-28-3,158.0,162.0,142.0,154.090730
3,2025-04-28-4,161.0,162.0,141.0,155.025620
4,2025-04-28-5,160.0,160.0,140.0,155.110200
...,...,...,...,...,...
2245,2025-07-01-46,109.0,113.0,93.0,110.052704
2246,2025-07-01-47,109.0,113.0,93.0,109.887670
2247,2025-07-01-48,109.0,113.0,93.0,109.707230
2248,2025-07-01-49,107.0,113.0,93.0,109.346500


In [3]:
df_predict["Volume"]=df_v["Volume"]
df_predict

,Unnamed: 0,Open,High,Low,Close,Volume
0,2025-04-28-1,158.0,160.0,140.0,152.052260,955
1,2025-04-28-2,152.0,161.0,144.0,152.442180,908
2,2025-04-28-3,158.0,162.0,142.0,154.090730,936
3,2025-04-28-4,161.0,162.0,141.0,155.025620,978
4,2025-04-28-5,160.0,160.0,140.0,155.110200,983
...,...,...,...,...,...,...
2245,2025-07-01-46,109.0,113.0,93.0,110.052704,1032
2246,2025-07-01-47,109.0,113.0,93.0,109.887670,1103
2247,2025-07-01-48,109.0,113.0,93.0,109.707230,1098
2248,2025-07-01-49,107.0,113.0,93.0,109.346500,1087


In [76]:
!pip install backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

class ExponentialMovingAverageStrategy(Strategy):
    def init(self, period=2, weighting_factor=0.1):
        self.period = period
        self.weighting_factor = weighting_factor
        self.ema_values = None

    def compute_ema(self, prices):
        ema = np.zeros(len(prices))
        if len(prices) >= self.period:
            sma = np.mean(prices[:self.period])
            ema[self.period - 1] = sma
            for i in range(self.period, len(prices)):
                ema[i] = (prices[i] * self.weighting_factor) + (ema[i - 1] * (1 - self.weighting_factor))
        return ema

    def next(self):
        self.ema_values = self.compute_ema(self.data['Close'])
        if self.data['Close'] > self.ema_values[-1]:  # Compare with the latest EMA value
            self.buy()
        else:
            self.sell()

In [77]:
bt = Backtest(df_predict, ExponentialMovingAverageStrategy,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
output

<ipython-input-77-9e0b16826625>:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_predict, ExponentialMovingAverageStrategy,


Start                                     0.0
End                                    2249.0
Duration                               2249.0
Exposure Time [%]                   72.133333
Equity Final [$]                       90.676
Equity Peak [$]                       10000.0
Return [%]                          -99.09324
Buy & Hold Return [%]              -29.408853
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -99.09324
Avg. Drawdown [%]                   -99.09324
Max. Drawdown Duration                 2248.0
Avg. Drawdown Duration                 2248.0
# Trades                               1620.0
Win Rate [%]                        45.185185
Best Trade [%]                      18.762475
Worst Trade [%]                    -19.627009
Avg. Trade [%]                    

In [73]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2638', ...)